<a href="https://colab.research.google.com/github/oktaviacitra/deep-metric-learning/blob/main/CIFAR10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

origin_path = "/content/drive"
drive.mount(origin_path)
folder_path = origin_path + "/MyDrive/Learning Journey/CIFAR10"

Mounted at /content/drive


In [2]:
from tensorflow.keras.datasets.cifar10 import load_data

(x_train, y_train), (x_test, y_test) = load_data()

170508288/170498071 [==============================] - 2s 0us/step


In [3]:
from tensorflow.image import per_image_standardization
from tensorflow.keras.utils import to_categorical

x_train, x_test = per_image_standardization(x_train).numpy(), per_image_standardization(x_test).numpy()
y_train, y_test = to_categorical(y_train), to_categorical(y_test)

In [4]:
x = x_train.copy()
y = y_train.copy()

In [6]:
!pip install -q -U keras-tuner

     |████████████████████████████████| 135 kB 4.7 MB/s 
     |████████████████████████████████| 1.6 MB 46.2 MB/s 


In [9]:
import keras_tuner as kt
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.applications.vgg16 import VGG16

class HyperModel(kt.HyperModel):
  def build(self, hp):
    model = VGG16(include_top=True,
                  weights=None,
                  input_shape=(32, 32, 3),
                  classes=10,
                  classifier_activation='softmax')
    
    learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4, 1e-5])
    optimizer = Adam(learning_rate=learning_rate) if hp.Choice('optimizer', ['adam', 'sgd']) == 'adam' else SGD(learning_rate=learning_rate)
    
    model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model
  
  def fit(self, hp, model, *args, **kwargs):
    return model.fit(*args,
                     batch_size=hp.Choice("batch_size", [16, 32]),
                     **kwargs,
                     )

In [10]:
from sklearn.model_selection import KFold
import numpy as np

kf = KFold(n_splits = 5)

VAL_ACC = []
VAL_LOSS = []
fold_count = 1


for train_indices, val_indices in kf.split(x,y):
  x_train = np.array([x[train_index] for train_index in train_indices])
  y_train = np.array([y[train_index] for train_index in train_indices])

  x_val = np.array([x[val_index] for val_index in val_indices])
  y_val = np.array([y[val_index] for val_index in val_indices])

  tuner = kt.RandomSearch(HyperModel(),
                          objective="val_loss",
                          max_trials=3,
                          overwrite=True,
                          directory=folder_path +"/dir",
                          project_name="tune_hypermodel",
                          )
  
  callbacks = [
      ModelCheckpoint(folder_path + "/tuner/{epoch:02d}-{val_loss:.4f}.h5",
                      monitor="val_loss",
                      mode="min",
                      save_best_only=True,
                      verbose=1),
      EarlyStopping(monitor='val_loss', patience=5)
  ]

  tuner.search(x_train, y_train, epochs=3,
              validation_data=(x_val, y_val),
              callbacks=callbacks,
              verbose=2)

Trial 2 Complete [00h 03m 13s]
val_loss: 2.3023946285247803

Best val_loss So Far: 2.3023946285247803
Total elapsed time: 00h 06m 37s

Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
0.01              |0.001             |learning_rate
sgd               |sgd               |optimizer
16                |16                |batch_size

Epoch 1/3



Epoch 1: val_loss improved from inf to 2.28327, saving model to /content/drive/MyDrive/Learning Journey/CIFAR10/tuner/01-2.2833.h5
2500/2500 - 64s - loss: 2.3004 - accuracy: 0.1112 - val_loss: 2.2833 - val_accuracy: 0.1848 - 64s/epoch - 26ms/step
Epoch 2/3

Epoch 2: val_loss improved from 2.28327 to 1.76169, saving model to /content/drive/MyDrive/Learning Journey/CIFAR10/tuner/02-1.7617.h5
2500/2500 - 63s - loss: 1.9808 - accuracy: 0.2431 - val_loss: 1.7617 - val_accuracy: 0.3250 - 63s/epoch - 25ms/step
Epoch 3/3


KeyboardInterrupt: ignored

https://medium.com/the-owl/k-fold-cross-validation-in-keras-3ec4a3a00538